In [96]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import scipy.special
import glob

pd.set_option('display.max_columns',85)
pd.set_option('display.max_rows',100)

In [103]:
# read in data to panda
# get the number of unclassified reads from Kraken2 outputs

#InPath="/home/isabel/Documents/postDoc_Amsterdam/1_EvolWormJourney/1_Genomics/2_EvolGenomics/2_Metagenomics/1_taxonomy/Bracken/withDB_standard/"
InPath="/home/isabel/Documents/postDoc_Amsterdam/1_EvolWormJourney/1_Genomics/2_EvolGenomics/2_Metagenomics/1_taxonomy/Bracken/withDB_plusPf/"
files=glob.glob(InPath+"*IR.genus.bracken")

OutName = "KraBracken_DBplusPf_062024_0.01minFrac_genus"
OutPath="/home/isabel/Documents/postDoc_Amsterdam/1_EvolWormJourney/1_Genomics/2_EvolGenomics/2_Metagenomics/1_taxonomy/"

threshold = 0.01
#files

In [104]:
# run a loop over all samples:
## -- extract the sample name from the file name 
## -- extract the data column wise
## -- merge to already existing df


for i in range(len(files)):

    # get sample name
    a = files[i].split('/')
    sample=a[-1].split('_')[0:5]
    sampleName='_'.join(sample)
    column = sampleName + "_abund"

    df_pre=pd.read_csv(files[i],header=0,delimiter="\t")
    #print(df_pre.shape)
    genus_here = df_pre['name']
    abund_here = df_pre['fraction_total_reads']

    d = {
        "genus": genus_here,
        column: abund_here
        }
    df_temp = pd.DataFrame(d)

    if i==0:
        df_merge = df_temp
    else:
        df_merge = pd.merge(df_merge, df_temp, on="genus", how="outer")

#threshold = 0.01
df_merge
df_collect = df_merge.fillna(0)

df_collect

,genus,VU_bn50_0_1_woCE_abund,VU_bn5_15_1_woCE_abund,WUR_bb_15_1_woCE_abund,WUR_bleach_15_1_woCE_abund,VU_bn50_0_5_woCE_abund,RUG_bb_1_4_woCE_abund,NIOO_M00_15_1_woCE_abund,VU_bn5_15_3_woCE_abund,NIOO_bb_15_4_woCE_abund,Gent_bb_15_3_woCE_abund,UVA_bb_0_1_woCE_abund,NIOO_A00_1_5_woCE_abund,VU_bn50_15_3_woCE_abund,Gent_bb_1_4_woCE_abund,NIOO_A00_1_2_woCE_abund,VU_bb_1_4_woCE_abund,VU_bn5_0_4_woCE_abund,WUR_bleach_15_2_woCE_abund,Gent_bb_15_4_woCE_abund,RUG_bb_15_3_woCE_abund,VU_bn5_0_2_woCE_abund,NIOO_A00_15_2_woCE_abund,VU_bb_0_5_woCE_abund,RUG_bb_15_5_woCE_abund,VU_bb_15_1_woCE_abund,RUG_bb_1_1_woCE_abund,NIOO_M00_1_5_woCE_abund,RUG_bb_15_1_woCE_abund,VU_bb_1_2_woCE_abund,WUR_bb_1_3_woCE_abund,VU_bn50_15_4_woCE_abund,VU_bn5_15_5_woCE_abund,NIOO_M00_15_5_woCE_abund,UVA_anc_0_3_woCE_abund,VU_bb_15_4_woCE_abund,WUR_bb_15_2_woCE_abund,RUG_bb_15_4_woCE_abund,NIOO_bb_15_3_woCE_abund,NIOO_M00_15_4_woCE_abund,RUG_bb_1_5_woCE_abund,VU_bn50_0_3_woCE_abund,...,UVA_anc_0_1_woCE_abund,VU_bn50_0_4_woCE_abund,Gent_bb_15_5_woCE_abund,NIOO_A00_1_4_woCE_abund,NIOO_A00_15_4_woCE_abund,Gent_bb_1_2_woCE_abund,NIOO_A00_1_3_woCE_abund,VU_bb_1_1_woCE_abund,NIOO_M00_1_3_woCE_abund,VU_bn5_0_1_woCE_abund,NIOO_A00_15_1_woCE_abund,NIOO_M00_1_4_woCE_abund,NIOO_bb_1_3_woCE_abund,VU_bb_1_3_woCE_abund,Gent_iso_15_2_woCE_abund,VU_bn50_15_2_woCE_abund,VU_bb_0_4_woCE_abund,VU_bn5_15_2_woCE_abund,RUG_bb_15_2_woCE_abund,VU_bn50_15_1_woCE_abund,VU_bn5_15_4_woCE_abund,Gent_bb_1_1_woCE_abund,NIOO_bb_15_1_woCE_abund,NIOO_A00_1_1_woCE_abund,VU_bn50_15_5_woCE_abund,VU_bb_15_2_woCE_abund,NIOO_A00_15_5_woCE_abund,VU_bn5_0_5_woCE_abund,VU_bn50_0_2_woCE_abund,VU_bb_0_1_woCE_abund,WUR_bb_1_1_woCE_abund,NIOO_bb_1_4_woCE_abund,WUR_bb_15_3_woCE_abund,VU_bn5_0_3_woCE_abund,RUG_bb_1_2_woCE_abund,Gent_bb_1_3_woCE_abund,NIOO_bb_1_1_woCE_abund,NIOO_M00_1_1_woCE_abund,Gent_iso_15_1_woCE_abund,Gent_iso_15_4_woCE_abund,Gent_iso_15_3_woCE_abund,UVA_anc_0_2_woCE_abund
0,Abyssalbus,0.00000,0.00001,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abyssibius,0.00005,0.00001,0.0,0.0,0.00016,0.0,0.0,0.00000,0.0,0.0,0.00002,0.0,0.0,0.0,0.0,0.0,0.00003,0.0,0.0,0.0,0.00005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00002,0.00001,0.0,0.0,0.00001,0.0,0.0,0.0,0.0,0.0,0.00001,...,0.0,0.00001,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00019,0.0,0.0,0.0,0.0,0.0,0.00001,0.0,0.0,0.0,0.00001,0.00001,0.0,0.0,0.0,0.00001,0.00001,0.0,0.00007,0.00005,0.0,0.0,0.0,0.0,0.00019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acaryochloris,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Acetilactobacillus,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Acetivibrio,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.

In [105]:
## exclude "homo" entry and renormalize the columns 

homo_loc = np.where(df_collect["genus"]=="Homo")[0]
df_collect_clean = df_collect.drop(homo_loc)
df_collect_clean.reset_index(drop=True, inplace=True)

# renormalize
for i in range(1,df_collect_clean.shape[1]):
    colName = df_collect_clean.columns[i]
    df_collect_clean[colName] = df_collect_clean[colName]/ np.sum(df_collect_clean[colName])
    
df_collect_clean

,genus,VU_bn50_0_1_woCE_abund,VU_bn5_15_1_woCE_abund,WUR_bb_15_1_woCE_abund,WUR_bleach_15_1_woCE_abund,VU_bn50_0_5_woCE_abund,RUG_bb_1_4_woCE_abund,NIOO_M00_15_1_woCE_abund,VU_bn5_15_3_woCE_abund,NIOO_bb_15_4_woCE_abund,Gent_bb_15_3_woCE_abund,UVA_bb_0_1_woCE_abund,NIOO_A00_1_5_woCE_abund,VU_bn50_15_3_woCE_abund,Gent_bb_1_4_woCE_abund,NIOO_A00_1_2_woCE_abund,VU_bb_1_4_woCE_abund,VU_bn5_0_4_woCE_abund,WUR_bleach_15_2_woCE_abund,Gent_bb_15_4_woCE_abund,RUG_bb_15_3_woCE_abund,VU_bn5_0_2_woCE_abund,NIOO_A00_15_2_woCE_abund,VU_bb_0_5_woCE_abund,RUG_bb_15_5_woCE_abund,VU_bb_15_1_woCE_abund,RUG_bb_1_1_woCE_abund,NIOO_M00_1_5_woCE_abund,RUG_bb_15_1_woCE_abund,VU_bb_1_2_woCE_abund,WUR_bb_1_3_woCE_abund,VU_bn50_15_4_woCE_abund,VU_bn5_15_5_woCE_abund,NIOO_M00_15_5_woCE_abund,UVA_anc_0_3_woCE_abund,VU_bb_15_4_woCE_abund,WUR_bb_15_2_woCE_abund,RUG_bb_15_4_woCE_abund,NIOO_bb_15_3_woCE_abund,NIOO_M00_15_4_woCE_abund,RUG_bb_1_5_woCE_abund,VU_bn50_0_3_woCE_abund,...,UVA_anc_0_1_woCE_abund,VU_bn50_0_4_woCE_abund,Gent_bb_15_5_woCE_abund,NIOO_A00_1_4_woCE_abund,NIOO_A00_15_4_woCE_abund,Gent_bb_1_2_woCE_abund,NIOO_A00_1_3_woCE_abund,VU_bb_1_1_woCE_abund,NIOO_M00_1_3_woCE_abund,VU_bn5_0_1_woCE_abund,NIOO_A00_15_1_woCE_abund,NIOO_M00_1_4_woCE_abund,NIOO_bb_1_3_woCE_abund,VU_bb_1_3_woCE_abund,Gent_iso_15_2_woCE_abund,VU_bn50_15_2_woCE_abund,VU_bb_0_4_woCE_abund,VU_bn5_15_2_woCE_abund,RUG_bb_15_2_woCE_abund,VU_bn50_15_1_woCE_abund,VU_bn5_15_4_woCE_abund,Gent_bb_1_1_woCE_abund,NIOO_bb_15_1_woCE_abund,NIOO_A00_1_1_woCE_abund,VU_bn50_15_5_woCE_abund,VU_bb_15_2_woCE_abund,NIOO_A00_15_5_woCE_abund,VU_bn5_0_5_woCE_abund,VU_bn50_0_2_woCE_abund,VU_bb_0_1_woCE_abund,WUR_bb_1_1_woCE_abund,NIOO_bb_1_4_woCE_abund,WUR_bb_15_3_woCE_abund,VU_bn5_0_3_woCE_abund,RUG_bb_1_2_woCE_abund,Gent_bb_1_3_woCE_abund,NIOO_bb_1_1_woCE_abund,NIOO_M00_1_1_woCE_abund,Gent_iso_15_1_woCE_abund,Gent_iso_15_4_woCE_abund,Gent_iso_15_3_woCE_abund,UVA_anc_0_2_woCE_abund
0,Abyssalbus,0.000000,0.00001,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Abyssibius,0.000066,0.00001,0.0,0.0,0.00016,0.0,0.0,0.00000,0.0,0.0,0.000021,0.0,0.0,0.0,0.0,0.0,0.00003,0.0,0.0,0.0,0.00005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00002,0.00001,0.0,0.0,0.00001,0.0,0.0,0.0,0.0,0.0,0.00001,...,0.0,0.00001,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00019,0.0,0.0,0.0,0.0,0.0,0.00001,0.0,0.0,0.0,0.00001,0.00001,0.0,0.0,0.0,0.00001,0.00001,0.0,0.00007,0.00005,0.0,0.0,0.0,0.0,0.00019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Acaryochloris,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Acetilactobacillus,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.0,0.00000,0.00000,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Acetivibrio,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,

In [106]:
# kick out rows that only contain 0:
## do this by first adding a column that includes the sum of samples that have this genus

rowNum = df_collect_clean.shape[0]

sumSamples = []
for i in range(rowNum):
    a  = np.array(df_collect_clean.iloc[i])[1:].astype(float)

    aa = np.count_nonzero(a > threshold)
    sumSamples.append(aa)

sumSamples = np.array(sumSamples)

# now only keep the rows that have a non zero entry

dropIdx = np.where(sumSamples == 0)[0]

df_nonZero = df_collect_clean.drop(dropIdx)
df_nonZero


df_nonZero.set_index('genus',inplace=True)

df_2 = df_nonZero.transpose()

df_2.shape


(93, 31)

In [107]:
# also, get the sample names as colums

institution = []
treatment = []
week = []
rep = []
newRowName = []

rowNames = df_2.index.tolist()

for i in rowNames:
    nameList = i.split('_')
    institution.append(nameList[0])
    treatment.append(nameList[1])
    week.append(nameList[2])
    rep.append(nameList[3])

    newRowName.append(nameList[0] + "_" + nameList[1] + "_" + nameList[2] + "_" + nameList[3])

#add columns to the dataframe
df_final=df_2
df_final.insert(0, "institution", institution)
df_final.insert(1, "treatment", treatment)
df_final.insert(2, "week", week)
df_final.insert(2, "replicate", rep)
df_final.insert(0, "name_short", newRowName)


genus,name_short,institution,treatment,replicate,week,Achromobacter,Agrobacterium,Bordetella,Brevundimonas,Brucella,Chryseobacterium,Debaryomyces,Epilithonimonas,Escherichia,Flavobacterium,Herbaspirillum,Kocuria,Leucobacter,Massilia,Microbacterium,Moraxella,Paenibacillus,Paracoccus,Plantibacter,Priestia,Pseudochrobactrum,Pseudomonas,Pseudoxanthomonas,Ralstonia,Rhodococcus,Roseomonas,Serratia,Sphingobacterium,Sphingomonas,Stenotrophomonas,Streptomyces
VU_bn50_0_1_woCE_abund,VU_bn50_0_1,VU,bn50,1,0,0.000052,0.001980,0.000026,0.008274,0.000092,0.021163,0.000000,0.003252,0.012942,0.000066,0.000013,0.014555,0.000144,0.000013,0.455583,0.007841,0.000013,0.138609,0.000013,0.000000,0.000000,0.000826,0.000013,0.000039,0.000223,0.308634,0.002072,0.000105,0.000393,0.000066,0.001351
VU_bn5_15_1_woCE_abund,VU_bn5_15_1,VU,bn5,1,15,0.011213,0.000731,0.007729,0.000080,0.000040,0.000491,0.000000,0.000010,0.001362,0.010572,0.000320,0.002383,0.000100,0.000350,0.272120,0.000010,0.000060,0.041427,0.014066,0.000100,0.000010,0.010702,0.000090,0.002953,0.000230,0.000060,0.000140,0.000280,0.000300,0.567107,0.000931
WUR_bb_15_1_woCE_abund,WUR_bb_15_1,WUR,bb,1,15,0.000040,0.000030,0.000000,0.000020,0.000000,0.000000,0.000000,0.000000,0.013324,0.000000,0.000020,0.008152,0.000300,0.000000,0.951977,0.000000,0.000020,0.000070,0.000040,0.000080,0.000000,0.000190,0.000000,0.000000,0.000930,0.000020,0.000010,0.000000,0.000060,0.000020,0.001810
WUR_bleach_15_1_woCE_abund,WUR_bleach_15_1,WUR,bleach,1,15,0.000040,0.000000,0.000000,0.000030,0.000090,0.000010,0.000000,0.000000,0.019823,0.000030,0.000010,0.007837,0.000301,0.000000,0.936574,0.000020,0.000020,0.000180,0.000040,0.001032,0.000040,0.000561,0.000010,0.000000,0.007025,0.000020,0.000351,0.000000,0.000070,0.001774,0.001794
VU_bn50_0_5_woCE_abund,VU_bn50_0_5,VU,bn50,5,0,0.000080,0.001962,0.000040,0.067302,0.000811,0.001181,0.000000,0.012752,0.009519,0.000040,0.000030,0.007677,0.000200,0.000060,0.622313,0.004664,0.000040,0.171608,0.000030,0.000010,0.000000,0.001662,0.042940,0.000070,0.000320,0.010289,0.000030,0.000110,0.004414,0.000400,0.001732
RUG_bb_1_4_woCE_abund,RUG_bb_1_4,RUG,bb,4,1,0.000050,0.000060,0.000000,0.000010,0.000020,0.000000,0.000000,0.000000,0.000130,0.000000,0.000010,0.003918,0.000140,0.000000,0.457374,0.000000,0.000020,0.000040,0.000020,0.009059,0.000010,0.004620,0.000020,0.000190,0.000210,0.000010,0.000020,0.000000,0.000030,0.510287,0.000892
NIOO_M00_15_1_woCE_abund,NIOO_M00_15_1,NIOO,M00,1,15,0.000080,0.001281,0.000020,0.000000,0.099354,0.000000,0.000000,0.000000,0.000190,0.000000,0.000000,0.000010,0.000010,0.000000,0.027025,0.000000,0.000010,0.000320,0.000000,0.000450,0.003262,0.009115,0.000020,0.000350,0.000010,0.000010,0.016739,0.000000,0.000050,0.832059,0.000080
VU_bn5_15_3_woCE_abund,VU_bn5_15_3,VU,bn5,3,15,0.007018,0.000581,0.004776,0.000060,0.000030,0.000340,0.000010,0.000010,0.000551,0.007119,0.000200,0.002623,0.000110,0.000230,0.299019,0.000000,0.000040,0.042962,0.006688,0.000050,0.000000,0.008680,0.000070,0.001912,0.000160,0.000040,0.000090,0.000190,0.000180,0.581628,0.000821
NIOO_bb_15_4_woCE_abund,NIOO_bb_15_4,NIOO,bb,4,15,0.000080,0.001002,0.000010,0.000000,0.044025,0.000010,0.000000,0.000000,0.000130,0.000010,0.000000,0.000431,0.000020,0.000000,0.050987,0.000000,0.000020,0.000210,0.000000,0.000160,0.010588,0.013784,0.000020,0.000321,0.000020,0.000010,0.044085,0.000000,0.000040,0.823919,0.000120
Gent_bb_15_3_woCE_abund,Gent_bb_15_3,Gent,bb,3,15,0.000060,0.000310,0.000040,0.000050,0.000010,0.000020,0.021629,0.000000,0.034227,0.000010,0.000120,0.002423,0.000421,0.000250,0.288522,0.000040,0.000030,0.000230,0.000020,0.000371,0.000000,0.000381,0.000010,0.000170,0.000541,0.000020,0.000020,0.000030,0.000120,0.000501,0.624740


In [108]:
# Sort and tidy up the dataframe
df_final=df_final.sort_values(['institution','treatment','week','replicate'])
df_final.shape

(93, 36)

In [109]:
df_final.to_csv(OutPath+OutName+"_FromPY.csv", index=False) #
#df2=pd.read_csv("data.csv")